https://www.mss.go.kr/site/smba/ex/bbs/List.do?cbIdx=310 의 링크에서 1501번째 글의 첨부파일을 다운받았음. 이것을 활용할 예정임

In [50]:
import os
from dotenv import load_dotenv
import openai
from PyPDF2 import PdfReader
import fitz  # pymupdf
import pdfplumber
import pdfminer.high_level
import pypdfium2 as pdfium

# 1. API 키 설정
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("API 키가 .env 파일에 설정되어 있지 않습니다.")

# 2. PDF 파일 경로
pdf_path = r"C:\Users\eys63\Desktop\기타활동\2024\겨울방학\24Winter_Vacation\data\2024년_중소기업_정책자금_융자공고(참고자료).pdf"

# 3. 각 라이브러리의 텍스트 추출 함수
def extract_with_pypdf(path):
    reader = PdfReader(path)
    return " ".join([page.extract_text() for page in reader.pages])

def extract_with_pymupdf(path):
    doc = fitz.open(path)
    return " ".join([page.get_text() for page in doc])

def extract_with_pdfplumber(path):
    with pdfplumber.open(path) as pdf:
        return " ".join([page.extract_text() for page in pdf.pages])

def extract_with_pdfminer(path):
    return pdfminer.high_level.extract_text(path)

def extract_with_pypdfium2(path):
    pdf = pdfium.PdfDocument(path)
    return " ".join([pdf[page].get_textpage().get_text_range() for page in range(len(pdf))])

# 4. 텍스트 500글자 단위로 분할
def split_text_by_chars(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# 5. OpenAI API를 사용한 평가 함수
def evaluate_with_openai(chunk_text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"""
            다음 텍스트 조각에 대해 다음과 같은 딕셔너리 형태의 결과를 반환해 주세요:
            {{"score": 숫자, "reason": "이유(한글)"}}

            - score는 0부터 100 사이의 숫자로 반환해주세요.
            - reason은 텍스트의 품질이나 내용에 대한 평가 이유를 한글로 작성해주세요.

            예시:
            1. {"score": 85, "reason": "문장이 명확하고 간결합니다."}
            2. {"score": 70, "reason": "내용이 다소 이해하기 어렵습니다."}
            3. {"score": 95, "reason": "내용이 매우 잘 구성되어 있으며 설득력이 있습니다."}

            다음 텍스트를 평가해 주세요:
            {chunk_text}
            """}
            ],
            max_tokens=10,
            temperature=0.5,
        )
        content = response['choices'][0]['message']['content'].strip()
        score = int(content.split(':')[1].strip())  # "Score: 85" 같은 형식 가정
        return score
    except Exception as e:
        return {"error": str(e)}

# 6. 모든 라이브러리 평가
libraries = {
    "pypdf": extract_with_pypdf,
    "pymupdf": extract_with_pymupdf,
    "pdfplumber": extract_with_pdfplumber,
    "pdfminer": extract_with_pdfminer,
    "pypdfium2": extract_with_pypdfium2,
}

scores = {}
for lib_name, extract_func in libraries.items():
    print(f"{lib_name}로 텍스트 추출 및 평가 시작...")
    try:
        text = extract_func(pdf_path)  # 텍스트 추출
        chunks = split_text_by_chars(text, chunk_size=500)  # 텍스트 분할
        lib_scores = []
        for chunk in chunks:
            # OpenAI API로 평가 요청
            result = evaluate_with_openai(chunk)
            if isinstance(result, dict) and "error" in result:
                print(f"{lib_name} 평가 중 오류 발생: {result['error']}")
                lib_scores.append(0)  # 오류 시 0점 처리
            else:
                lib_scores.append(result["score"])  # 딕셔너리에서 score 값만 사용
        avg_score = sum(lib_scores) / len(lib_scores) if lib_scores else 0
        scores[lib_name] = avg_score
        print(f"{lib_name} 평가 완료: 평균 점수 {avg_score:.2f}")
    except Exception as e:
        print(f"{lib_name}에서 오류 발생: {e}")
        scores[lib_name] = 0

# 점수 저장 함수
def save_scores_as_md(scores, output_md_path="score.md"):
    try:
        with open(output_md_path, "w") as f:
            f.write("# PDF Library Evaluation Scores\n\n")
            for lib_name, score in scores.items():
                f.write(f"- **{lib_name.capitalize()}**: {score:.2f}\n")
        print(f"Scores successfully saved to {output_md_path}")
    except Exception as e:
        print(f"Error while saving scores: {e}")

# 점수 저장 실행
save_scores_as_md(scores)


pypdf로 텍스트 추출 및 평가 시작...
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85, "reason": "문장이 명확하고 간결합니다."' for object of type 'str'
pypdf 평가 중 오류 발생: Invalid format specifier ' 85,

KeyboardInterrupt: 